<a href="https://colab.research.google.com/github/Above02/Statistical_Analysis/blob/master/Machine_Learning/code/Pytorch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<h1>Pytorch</h1>
---

El aprendizaje profundo ha reavivado el interés público por la IA. 

La razón es simple: el aprendizaje profundo simplemente funciona. Nos ha dado la capacidad de crear tecnologías que antes no podíamos. Ha creado nuevas oportunidades comerciales y mejorado el mundo de la tecnología en su conjunto.

---
Para hacer Deep Learning, necesitaremos saber cómo programar, especialmente con Python. A partir de ahí, hay un gran conjunto cada vez mayor de bibliotecas de aprendizaje profundo para elegir: TensorFlow, Keras, MXNet, MatConvNet y, recientemente, Pytorch.

Muy poco después de su lanzamiento, Pytorch ganó popularidad rápidamente. La gente lo llamaba el asesino de TensorFlow, ya que era mucho más amigable con el usuario y fácil de usar. 

---
De hecho, veremos un breve ejemplo de lo fácil que es empezar a trabajar en Deep Learning con Pytorch.



In [ ]:
!pip install torch torchvision

### Tensores

El bloque de construcción más básico de cualquier biblioteca de Deep Learning es el tensor. 

---
Los tensores son estructuras de datos en forma de matriz muy similares en función y propiedades a las matrices Numpy. De hecho, para la mayoría de los propósitos, podemos pensar en ellos exactamente como matrices Numpy. La diferencia más importante entre los dos es que la implementación de tensores en las bibliotecas modernas de Deep Learning puede ejecutarse en CPU o GPU (muy rápido).

In [ ]:
import torch 
x = torch.Tensor(3, 3)
print(x)

tensor([[6.3695e-36, 0.0000e+00, 3.7835e-44],
        [0.0000e+00,        nan, 0.0000e+00],
        [1.3733e-14, 6.4069e+02, 4.3066e+21]])


También podemos crear tensores llenos de valores de coma flotante aleatorios:

In [ ]:
x = torch.rand(3, 3)
print(x)

tensor([[0.3003, 0.3393, 0.3516],
        [0.2213, 0.4582, 0.6519],
        [0.1333, 0.1241, 0.3625]])


In [ ]:
x = torch.ones(3,3)
y = torch.ones(3,3) * 4
z = x + y
print(z)

tensor([[5., 5., 5.],
        [5., 5., 5.],
        [5., 5., 5.]])


¡Incluso las funciones de corte similares a Numpy están disponibles con los tensores Pytorch!

In [ ]:
x = torch.ones(3,3) * 5
y = x[:, :2]
print(y)

tensor([[5., 5.],
        [5., 5.],
        [5., 5.]])


Por lo tanto, los tensores Pytorch se pueden usar y trabajar de la misma manera que las matrices Numpy. 

Ahora veremos cómo podemos construir Deep Networks con estos tensores Pytorch fáciles como nuestros bloques de construcción.

---

### Construyendo redes neuronales con Pytorch

Con Pytorch, las redes neuronales se definen como clases de Python. La clase que define la red extiende el módulo torch.nn. de la biblioteca Torch. 


Creemos una clase para una red neuronal convolucional (CNN) que aplicaremos en el conjunto de datos MNIST.

In [ ]:
import torch.nn as nn
import torch.nn.functional as F
import torch

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 64, kernel_size=(3, 3), padding=1)
        self.conv2 = nn.Conv2d(64, 64, kernel_size=(3, 3), padding=1)
        self.max_pool = nn.MaxPool2d(2, 2)
        self.global_pool = nn.AvgPool2d(7)
        self.fc1 = nn.Linear(64, 64)
        self.fc2 = nn.Linear(64, 10)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = F.relu(self.conv2(x))
        x = self.max_pool(x)

        x = F.relu(self.conv2(x))
        x = F.relu(self.conv2(x))
        x = self.max_pool(x)

        x = F.relu(self.conv2(x))
        x = F.relu(self.conv2(x))
        x = self.global_pool(x)

        x = x.view(-1, 64)

        x = F.relu(self.fc1(x))
        x = self.fc2(x)

        x = F.log_softmax(x)

        return x
model = Net()

Las dos funciones más importantes en una clase de red Pytorch son las funciones __init __ () y forward (). El __init __ () se usa para definir las capas de red que usará su modelo. La función forward () es donde realmente configura el modelo apilando todas las capas juntas.

Para nuestro modelo, hemos definido 2 capas convolucionales en la función init, una de las cuales la reutilizaremos varias veces (conv2). Tenemos una capa de agrupación máxima y una capa de agrupación promedio global que se aplicará cerca del final. Finalmente tenemos nuestras capas Full-Connected (FC) y un softmax para obtener las probabilidades de salida finales.

En la función de avance, definimos exactamente cómo se apilan nuestras capas para formar el modelo completo. Es una red estándar con capas de conversión, agrupación y FC apiladas. La belleza de Pytorch es que podemos imprimir la forma y el resultado de cualquier tensor dentro de las capas intermedias con solo una simple declaración de impresión donde lo desee en la función forward ().

### Entrenamiento, pruebas y Guardado

#### Cargando datos

Es hora de preparar nuestros datos para el entrenamiento...

Comenzaremos, pero prepararemos las importaciones necesarias, inicializaremos los parámetros y nos aseguraremos de que Pytorch esté configurado para usar la GPU. La siguiente línea que usa torch.device () verifica si Pytorch se instaló con soporte CUDA y, si es así, usa la GPU.

In [3]:
import torch
import torchvision
import torchvision.transforms as transforms

num_epochs = 10
batch_size = 32
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [4]:
device

device(type='cpu')

Podemos recuperar el conjunto de datos MNIST directamente de Pytorch. 

Descargaremos los datos y colocaremos los conjuntos de entrenamiento y de prueba en tensores separados. Una vez que se cargan esos datos, los pasaremos a un DataLoader de torch que simplemente los prepara para pasar al modelo con un tamaño de lote específico y barajado opcional.


In [ ]:
# MNIST dataset
train_dataset = torchvision.datasets.MNIST(root='data',
                                           train=True, 
                                           transform=transforms.ToTensor(),
                                           download=True)

test_dataset = torchvision.datasets.MNIST(root='data',
                                          train=False,
                                          transform=transforms.ToTensor())

# Data loader
train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                           batch_size=batch_size,
                                           shuffle=True)

test_loader = torch.utils.data.DataLoader(dataset=test_dataset,
                                          batch_size=batch_size,
                                          shuffle=False)

Extracting data/MNIST/raw/train-images-idx3-ubyte.gz to data/MNIST/raw


Extracting data/MNIST/raw/train-labels-idx1-ubyte.gz to data/MNIST/raw


Extracting data/MNIST/raw/t10k-images-idx3-ubyte.gz to data/MNIST/raw




Extracting data/MNIST/raw/t10k-labels-idx1-ubyte.gz to data/MNIST/raw
Processing...
Done!


/usr/local/lib/python3.6/dist-packages/torchvision/datasets/mnist.py:480: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  /pytorch/torch/csrc/utils/tensor_numpy.cpp:141.)
  return torch.from_numpy(parsed.astype(m[2], copy=False)).view(*s)


### Es hora de entrenar

El optimizador (usaremos Adam) y la función de pérdida (usaremos entropía cruzada) se definen de manera bastante similar a otras bibliotecas de aprendizaje profundo como TensorFlow, Keras y MXNet.

In [ ]:
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
loss_function = nn.CrossEntropyLoss()



En Pytorch, todos los modelos de red y conjuntos de datos se transfieren explícitamente de la CPU a la GPU. 

Hacemos esto aplicando la función .to () a nuestro modelo a continuación.

 Más adelante, haremos lo mismo con nuestros datos de imagen.

In [ ]:
model.to(device)

Net(
  (conv1): Conv2d(1, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (max_pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (global_pool): AvgPool2d(kernel_size=7, stride=7, padding=0)
  (fc1): Linear(in_features=64, out_features=64, bias=True)
  (fc2): Linear(in_features=64, out_features=10, bias=True)
)

Finalmente, podemos escribir nuestro ciclo de entrenamiento. 

    Todos los ciclos de entrenamiento de Pytorch pasarán por cada época y cada lote en el cargador de datos de entrenamiento.
    En cada iteración de bucle, los datos de la imagen y las etiquetas se transfieren a la GPU.
    Cada ciclo de entrenamiento también aplica explícitamente los pasos de paso hacia adelante, paso hacia atrás y optimización.
    El modelo se aplica a las imágenes en el lote y luego se calcula la pérdida para ese lote.
    Los gradientes se calculan y se propagan a través de la red.

In [ ]:
# Train the model
total_step = len(train_loader)
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
        # Move tensors to the configured device
        images = images.to(device)
        labels = labels.to(device)

        # Forward pass
        outputs = model(images)
        loss = loss_function(outputs, labels)

        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if (i+1) % 100 == 0:
            print ('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}'
                   .format(epoch+1, num_epochs, i+1, total_step, loss.item()))

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:33: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.


Epoch [1/10], Step [100/1875], Loss: 2.3103
Epoch [1/10], Step [200/1875], Loss: 1.7507
Epoch [1/10], Step [300/1875], Loss: 0.6271
Epoch [1/10], Step [400/1875], Loss: 0.3302
Epoch [1/10], Step [500/1875], Loss: 0.4663
Epoch [1/10], Step [600/1875], Loss: 0.2029
Epoch [1/10], Step [700/1875], Loss: 0.3614
Epoch [1/10], Step [800/1875], Loss: 0.2634
Epoch [1/10], Step [900/1875], Loss: 0.2797
Epoch [1/10], Step [1000/1875], Loss: 0.1772
Epoch [1/10], Step [1100/1875], Loss: 0.0226
Epoch [1/10], Step [1200/1875], Loss: 0.0640
Epoch [1/10], Step [1300/1875], Loss: 0.2379
Epoch [1/10], Step [1400/1875], Loss: 0.0254
Epoch [1/10], Step [1500/1875], Loss: 0.1616
Epoch [1/10], Step [1600/1875], Loss: 0.2312
Epoch [1/10], Step [1700/1875], Loss: 0.0141
Epoch [1/10], Step [1800/1875], Loss: 0.0569
Epoch [2/10], Step [100/1875], Loss: 0.0344
Epoch [2/10], Step [200/1875], Loss: 0.2010
Epoch [2/10], Step [300/1875], Loss: 0.2821
Epoch [2/10], Step [400/1875], Loss: 0.0119
Epoch [2/10], Step [500

### Prueba y Guardado

Probar el rendimiento de una red en Pytorch establece un ciclo similar al de la fase de entrenamiento. 

La principal diferencia es que no necesitamos hacer una propagación hacia atrás de los gradientes. 

Seguiremos haciendo el pase directo y solo obtendremos la etiqueta con la máxima probabilidad en la salida de la red.

En este caso, después de 10 épocas, nuestra red obtuvo una precisión del 99,06% en el equipo de prueba.



In [ ]:
# Test the model
# In test phase, we don't need to compute gradients (for memory efficiency)
with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in test_loader:
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    print('Accuracy of the network on the MNIST test images: {} %'.format(100 * correct / total))

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:33: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.


Accuracy of the network on the MNIST test images: 99.17 %


In [ ]:
torch.save(model.state_dict(), 'model.ckpt')

### Transferencia de aprendizaje para el aprendizaje profundo

----------

El aprendizaje por transferencia es un método de aprendizaje automático en el que un modelo desarrollado para una tarea se reutiliza como punto de partida para un modelo en una segunda tarea.

Es un enfoque popular en el aprendizaje profundo en el que los modelos previamente entrenados se utilizan como punto de partida en la visión por computadora y las tareas de procesamiento del lenguaje natural, dada la gran cantidad de recursos informáticos y de tiempo necesarios para desarrollar modelos de redes neuronales en estos problemas y a partir de los enormes saltos en las habilidades. que proporcionan sobre problemas relacionados.

----

### ¿Qué es el aprendizaje por transferencia?

El aprendizaje por transferencia es una técnica de aprendizaje automático en la que un modelo entrenado en una tarea se reorienta en una segunda tarea relacionada.

---

Esta forma de aprendizaje por transferencia utilizada en el aprendizaje profundo se denomina **transferencia inductiva**. Aquí es donde el alcance de los posibles modelos (sesgo del modelo) se reduce de una manera beneficiosa al utilizar un modelo que se ajusta a una tarea diferente pero relacionada.

<img src="https://3qeqpr26caki16dnhd19sv6by6v-wpengine.netdna-ssl.com/wp-content/uploads/2017/09/Depiction-of-Inductive-Transfer.png"/>

### ¿Cómo utilizar el aprendizaje transferido?

Puedes utilizar el aprendizaje por transferencia en tus propios problemas de modelado predictivo.

Dos enfoques comunes son los siguientes:

### Model Approach - Desarrollo

* **Seleccionar Tarea de origen**: Debes seleccionar un problema de modelado predictivo relacionado con una gran cantidad de datos donde existe alguna relación en los datos de entrada, datos de salida y / o conceptos aprendidos durante el mapeo de datos de entrada a salida.
* **Desarrollar modelo fuente**: A continuación, debes desarrollar un modelo hábil para esta primera tarea. El modelo debe ser mejor que un modelo ingenuo para garantizar que se haya realizado algún aprendizaje de características.
* **Modelo reutilizado**: El modelo que se ajusta a la tarea de origen se puede utilizar como punto de partida para un modelo en la segunda tarea de interés. Esto puede implicar el uso total o parcial del modelo, dependiendo de la técnica de modelado utilizada.
* **Refinar modelo**. Opcionalmente, el modelo puede necesitar ser adaptado o refinado en los datos del par de entrada-salida disponibles para la tarea de interés.

---

### Model Approach - Pre-entrenamiento

* **Seleccionar el modelo de origen**. Se elige un modelo de origen entrenado previamente entre los modelos disponibles. Muchas instituciones de investigación publican modelos en conjuntos de datos grandes y desafiantes que pueden incluirse en el grupo de modelos candidatos entre los que elegir.
* **Modelo reutilizado**. El modelo previamente entrenado se puede utilizar como punto de partida para un modelo en la segunda tarea de interés. Esto puede implicar el uso total o parcial del modelo, dependiendo de la técnica de modelado utilizada.
* **Refinar modelo**. Opcionalmente, el modelo puede necesitar ser adaptado o refinado en los datos del par de entrada-salida disponibles para la tarea de interés.

---
---



### Ejemplos de transferencia de aprendizaje con aprendizaje profundo

Hagamos esto concreto con dos ejemplos comunes de transferencia de aprendizaje con modelos de aprendizaje profundo.

#### **Transferir aprendizaje con datos de imagen**

Es común realizar el aprendizaje por transferencia con problemas de modelado predictivo que utilizan datos de imagen como entrada.

Esta puede ser una tarea de predicción que toma fotografías o datos de video como entrada.

Para este tipo de problemas, es común utilizar un modelo de aprendizaje profundo previamente entrenado para una tarea de clasificación de imágenes grande y desafiante, como la competencia de clasificación de fotografías de la clase ImageNet1000.

Las organizaciones de investigación que desarrollan modelos para esta competencia y lo hacen bien a menudo publican su modelo final bajo una licencia permisiva para su reutilización. **Estos modelos pueden tardar días o semanas en entrenarse con hardware moderno**.

Estos modelos se pueden [descargar](https://pytorch.org/docs/stable/torchvision/models.html) e incorporar directamente en nuevos modelos que esperan datos de imagen como entrada.
---

Este enfoque es efectivo porque las imágenes se entrenaron en un gran corpus de fotografías y requieren que el modelo haga predicciones en un número relativamente grande de clases, lo que a su vez requiere que el modelo aprenda de manera eficiente a extraer características de fotografías para funcionar bien en el problema.

En su curso de Stanford sobre redes neuronales convolucionales para el reconocimiento visual, los autores advierten que deben elegir cuidadosamente qué cantidad del modelo previamente entrenado usar en su nuevo modelo.

---
---

#### **Transferir el aprendizaje con datos lingüísticos**

Es común realizar el aprendizaje por transferencia con problemas de procesamiento del lenguaje natural que utilizan texto como entrada o salida.

Para este tipo de problemas, se utiliza una incrustación de palabras que es un mapeo de palabras a un espacio vectorial continuo de alta dimensión donde diferentes palabras con un significado similar tienen una representación vectorial similar.

Existen algoritmos eficientes para aprender estas representaciones de palabras distribuidas y es común que las organizaciones de investigación publiquen modelos previamente entrenados entrenados en corpa muy grande de documentos de texto bajo una licencia permisiva.

* Estos modelos de representación de palabras distribuidas se pueden descargar e incorporar en modelos de lenguaje de aprendizaje profundo, ya sea en la interpretación de palabras como entrada o en la generación de palabras como salida del modelo.






---
---

### ¿Cuándo utilizar el aprendizaje transferido?

La transferencia de aprendizaje es una optimización, un atajo para ahorrar tiempo o obtener un mejor rendimiento.

En general, no es obvio que habrá un beneficio al utilizar el aprendizaje por transferencia en el dominio hasta que se haya desarrollado y evaluado el modelo.

---

Tres posibles beneficios a tener en cuenta al utilizar el aprendizaje por transferencia:

* Comienzo superior. La habilidad inicial (antes de refinar el modelo) en el modelo de origen es mayor de lo que sería de otra manera.
* Pendiente más alta. La tasa de mejora de la habilidad durante el entrenamiento del modelo fuente es mayor de lo que sería de otra manera.
* Asíntota superior. La habilidad convergente del modelo entrenado es mejor de lo que sería de otra manera.

<img src="https://3qeqpr26caki16dnhd19sv6by6v-wpengine.netdna-ssl.com/wp-content/uploads/2017/09/Three-ways-in-which-transfer-might-improve-learning-1024x505.png"/>

Idealmente, estos serían los tres beneficios de una aplicación exitosa del aprendizaje por transferencia.

Es un enfoque para probar si pueden identificar una tarea relacionada con datos abundantes y tienes los recursos para desarrollar un modelo para esa tarea y reutilizarlo en su propio problema, o si hay un modelo previamente entrenado disponible que puede usar como un punto de partida para su propio modelo.

**En algunos problemas en los que es posible que no tenga muchos datos, el aprendizaje por transferencia puede permitirle desarrollar modelos hábiles que simplemente no podría desarrollar en ausencia del aprendizaje por transferencia.**

La elección de la fuente de datos o el modelo de fuente es un problema abierto y puede requerir conocimientos de dominio y / o intuición desarrollada a través de la experiencia.